In [ ]:
# Standard imports
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import MinMaxScaler

# === GPU Check ===
print("🔍 GPU Status:")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"   ✅ Found {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}")
    # Enable memory growth to avoid OOM
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("   ⚠️ No GPU found - training will be slow!")
    print("   In Colab: Runtime → Change runtime type → GPU")

# === Mixed Precision: BFloat16 (A100 optimized) ===
# BFloat16 has same dynamic range as Float32 (8-bit exponent)
# - No loss scaling required
# - Stable for recurrent networks (eliminates NaN issue)
# - Native Tensor Core acceleration on A100
from tensorflow.keras import mixed_precision
if gpus:
    try:
        mixed_precision.set_global_policy('mixed_bfloat16')
        print(f"\n✅ Mixed Precision: mixed_bfloat16 (A100 optimized)")
    except Exception as e:
        print(f"⚠️ BFloat16 not available ({e}), falling back to float16")
        mixed_precision.set_global_policy('mixed_float16')
    policy = mixed_precision.global_policy()
    print(f"   Compute dtype: {policy.compute_dtype}")
    print(f"   Variable dtype: {policy.variable_dtype}")
else:
    print("\n⚠️ Mixed Precision skipped (no GPU)")

In [ ]:
# environment configuration
# check if running in colab

if "google.colab" in sys.modules:
  from google.colab import drive
  print("Running in Colab")
  drive.mount('/content/drive')

  # EDIT THIS: Your exact folder path in Drive
  PROJECT_ROOT = "/content/drive/MyDrive/Colab Notebooks/headway-prediction"
else:
    print("💻 Running in Local Environment")
    # Assuming notebook is in /notebooks, root is one level up
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# system setup path
if PROJECT_ROOT not in sys.path:
  sys.path.append(PROJECT_ROOT)
  print(f"added to sys.path: {PROJECT_ROOT}")

print(f"Project Root: {PROJECT_ROOT}")

Running in Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
added to sys.path: /content/drive/MyDrive/Colab Notebooks/headway-prediction
Project Root: /content/drive/MyDrive/Colab Notebooks/headway-prediction


In [ ]:
# Validate Imports
try:
    from src.config import Config
    from src.data.dataset import SubwayDataGenerator
    from src.models.convlstm import ConvLSTM  # Class-based architecture
    from src.training.trainer import Trainer
    from src.evaluator import Evaluator
    print("✅ Success: All custom 'src' modules imported.")
except ImportError as e:
    print(f"❌ IMPORT ERROR: {e}")
    raise

✅ Success: All custom 'src' modules imported.


# Data Loading

In [ ]:
# Data loading and scaling
# Paper uses MinMax normalization to [0,1] (Section 3.1)

config = Config()
config.DATA_DIR = os.path.join(PROJECT_ROOT, "data")
print(f"Loading data from {config.DATA_DIR}")

# 1. Instantiate data generator
data_gen = SubwayDataGenerator(config)

# 2. Load raw .npy files (no normalization - we'll use MinMaxScaler per paper)
data_gen.load_data(normalize=False)
print(f"Raw max headway values: {data_gen.headway_data.max():.2f} min")

# 3. Fit MinMaxScaler (Paper Section 3.1: "normalized to [0,1] using min-max scaling")
total_timesteps = len(data_gen.headway_data)
train_limit = int(total_timesteps * 0.6)

print(f"\n📄 Paper: 'headway values normalized to the interval [0, 1] using min-max scaling'")
print(f"Fitting MinMaxScaler on first {train_limit} steps")
scaler = MinMaxScaler(feature_range=(0, 1))
flat_train = data_gen.headway_data[:train_limit].reshape(-1, 1)
scaler.fit(flat_train)

# 4. Transform All Data
print("Transforming Headway and Schedule Data")
data_gen.headway_data = scaler.transform(data_gen.headway_data.reshape(-1, 1)).reshape(data_gen.headway_data.shape)
data_gen.schedule_data = scaler.transform(data_gen.schedule_data.reshape(-1, 1)).reshape(data_gen.schedule_data.shape)

# 5. Save scaler for inference
scaler_path = os.path.join(PROJECT_ROOT, "models", "minmax_scaler.pkl")
os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to {scaler_path}")

print(f"\nScaled data range: [{data_gen.headway_data.min():.4f}, {data_gen.headway_data.max():.4f}]")

Loading data from /content/drive/MyDrive/Colab Notebooks/headway-prediction/data
Loading data from /content/drive/MyDrive/Colab Notebooks/headway-prediction/data...
Headway Shape: (264222, 66, 2, 1)
Schedule Shape: (264222, 2, 1)
Raw max headway values: 30.0 min (should be ~30.0)

Fitting RobustScaler on first 158533 steps
Transforming Headway and Schedule Data
Scaler saved to /content/drive/MyDrive/Colab Notebooks/headway-prediction/models/robust_scaler.pkl


# Baseline Experiment Configuration

In [ ]:
# Configuration
# Large batch size to saturate Tensor Cores and reduce kernel launch overhead
# BFloat16 + unroll=True should stabilize training

config.LOOKBACK_MINS = 30
config.FORECAST_MINS = 15
config.BATCH_SIZE = 256  # Large batch for A100 throughput (amortizes fixed overhead)
config.EPOCHS = 100
config.LEARNING_RATE = 0.001  # May need to scale with batch size

print(f'--- Configuration ---')
print(f'Lookback: {config.LOOKBACK_MINS} minutes')
print(f'Forecast: {config.FORECAST_MINS} minutes')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Epochs: {config.EPOCHS}')
print(f'Learning Rate: {config.LEARNING_RATE}')

# Create tf datasets (60% train, 20% val, 20% test)
train_end = int(0.6 * total_timesteps)
val_end = int(0.8 * total_timesteps)

print(f"\nCreating datasets...")
train_ds = data_gen.make_dataset(start_index=0, end_index=train_end, shuffle=True)
val_ds = data_gen.make_dataset(start_index=train_end, end_index=val_end, shuffle=False)
test_ds = data_gen.make_dataset(start_index=val_end, end_index=None, shuffle=False)

# Shape verification (critical sanity check!)
print("\n🔍 Shape Verification:")
for inputs, targets in train_ds.take(1):
    print(f"   headway_input:  {inputs['headway_input'].shape}  (expected: [batch, 30, 66, 2])")
    print(f"   schedule_input: {inputs['schedule_input'].shape}  (expected: [batch, 15, 2])")
    print(f"   target:         {targets.shape}  (expected: [batch, 15, 66, 2])")
    
    # Validate shapes match model expectations
    assert inputs['headway_input'].shape[1:] == (30, 66, 2), "❌ headway_input shape mismatch!"
    assert inputs['schedule_input'].shape[1:] == (15, 2), "❌ schedule_input shape mismatch!"
    assert targets.shape[1:] == (15, 66, 2), "❌ target shape mismatch!"
    print("   ✅ All shapes verified!")

--- Baseline Run Config --- 
lookback: 30
batch: 64
epochs: 20
filters: 64

creating datasets...
Creating dataset from index 0 to 158533
Creating dataset from index 158533 to 211377
Creating dataset from index 211377 to 264177
Input headway shape: (64, 30, 66, 2, 1)
Target shape: (64, 15, 66, 2, 1)


# Model Build and Training

In [ ]:
# Build and train using the Trainer module
print(f"\n🏗️ Building Model...")

# Use ConvLSTM encoder-decoder architecture
model_builder = ConvLSTM(config)
model = model_builder.build_model()
model.summary()

# Calculate steps per epoch for CosineDecay LR schedule
steps_per_epoch = train_end // config.BATCH_SIZE
print(f"\nSteps per epoch: {steps_per_epoch}")

# Use Trainer class for clean compilation and training
checkpoint_dir = os.path.join(PROJECT_ROOT, "models")
trainer = Trainer(model, config, checkpoint_dir=checkpoint_dir, steps_per_epoch=steps_per_epoch)
trainer.compile_model()

print("\n🚀 Starting Training...")
history = trainer.fit(
    train_ds, 
    val_ds,
    patience=20,  # Early stopping patience (paper uses 50)
    reduce_lr_patience=7  # Reduce LR if no improvement for 7 epochs
)

print(f"✅ Training complete")


Building V2 Architecture
Starting Training...
Epoch 1/20
2477/2477 ━━━━━━━━━━━━━━━━━━━━ 499s 197ms/step - loss: 0.1695 - mae: 0.4089 - mse: 0.3959 - val_loss: 0.1649 - val_mae: 0.3905 - val_mse: 0.3955 - learning_rate: 0.0010
Epoch 2/20
 145/2477 ━━━━━━━━━━━━━━━━━━━━ 6:46 175ms/step - loss: 0.1515 - mae: 0.3731 - mse: 0.3572

# Model Evaluation

In [ ]:
# Evaluate best model on independent TEST set
# This is the held-out 20% that was never seen during training or validation
from src.metrics import rmse_seconds, r_squared

print("=" * 60)
print("🧪 FINAL TEST SET EVALUATION (Independent Data)")
print("=" * 60)

# Load best model from checkpoint using ConvLSTM's class method
best_model_path = os.path.join(PROJECT_ROOT, "models", "best_model.keras")
print(f"\nLoading best model from: {best_model_path}")
best_model = ConvLSTM.load_model(best_model_path)

# Evaluate on test set
print("\nEvaluating on test set...")
test_results = best_model.evaluate(test_ds, verbose=0)

# Extract metrics (order: loss, rmse_seconds, r_squared)
test_loss = test_results[0]
test_rmse_sec = test_results[1]
test_r2 = test_results[2]

# Also compute RMSE from loss for verification
test_rmse_from_loss = np.sqrt(test_loss) * (scaler.data_max_[0] - scaler.data_min_[0]) * 60

print(f"\n📊 Test Set Results:")
print(f"   RMSE: {test_rmse_sec:.1f} seconds ({test_rmse_sec/60:.2f} minutes)")
print(f"   R²:   {test_r2:.4f}")

# Production readiness on TEST data
print(f"\n🚦 Production Readiness (on unseen test data):")
if test_rmse_sec <= 60:
    print(f"   ✅ EXCELLENT - {test_rmse_sec:.1f}s RMSE suitable for real-time displays")
elif test_rmse_sec <= 90:
    print(f"   ✅ GOOD - {test_rmse_sec:.1f}s RMSE suitable for trip planning")
elif test_rmse_sec <= 120:
    print(f"   ⚠️ ACCEPTABLE - {test_rmse_sec:.1f}s RMSE for general information")
else:
    print(f"   ❌ NEEDS WORK - {test_rmse_sec:.1f}s RMSE exceeds production threshold")

if test_r2 >= 0.95:
    print(f"   ✅ R² = {test_r2:.4f} - Excellent explanatory power")
elif test_r2 >= 0.90:
    print(f"   ✅ R² = {test_r2:.4f} - Good explanatory power")
else:
    print(f"   ⚠️ R² = {test_r2:.4f} - Consider model improvements")

In [ ]:
# Training curves and paper-style visualizations
evaluator = Evaluator(config, scaler=scaler)

# Save directory for plots
save_dir = os.path.join(PROJECT_ROOT, "images")
os.makedirs(save_dir, exist_ok=True)

# 1. Training curves (RMSE and R²)
print("\n📈 Training Curves:")
evaluator.plot_training_curves(history, save_path=os.path.join(save_dir, "training_curves.png"))

# 2. Paper-style heatmap visualizations (Figure 7 style)
print("\n🗺️ Spatiotemporal Prediction Visualization:")
evaluator.plot_spatiotemporal_prediction(
    best_model, test_ds, 
    sample_idx=0, direction=0,
    save_path=os.path.join(save_dir, "prediction_northbound.png")
)
evaluator.plot_spatiotemporal_prediction(
    best_model, test_ds, 
    sample_idx=0, direction=1,
    save_path=os.path.join(save_dir, "prediction_southbound.png")
)